# Kerchunk Usage

### Imports

In [ ]:
import fsspec
from kerchunk.combine import MultiZarrToZarr
from kerchunk.hdf import SingleHdf5ToZarr
import os
from pathlib import Path
import shutil
import ujson
import xarray as xr

from tqdm import tqdm # Optional: Progress bar for potentially long running operations

import time

## Paths

In [ ]:
S3_PREFIX = 's3://landscapes-easi-shared/misc/demo/himawari/lst'
LOCAL_DIR = '/tmp'

## Single data file indexing and use

### Creating a single index JSON for a single data file

In [ ]:
# https://fsspec.github.io/kerchunk/reference.html

# Example: Data file is stored in AWS S3, index JSON will be created locally.

data_file_url = S3_PREFIX + "/2016/01/01/20160101000000_AHI_ANU_LSTv1.4.1_AusSubset.nc"
index_json_file_path = os.path.join(LOCAL_DIR, "single_file_kerchunk.json")

fs = fsspec.filesystem("s3") # Can pass in additional args for authentication to S3 if required here.

with fs.open(data_file_url) as inf:
    # Generate index JSON
    h5chunks = SingleHdf5ToZarr(inf, data_file_url, inline_threshold=100)
    h5chunks.translate()
    # Write index JSON to file
    with open(index_json_file_path, "wb") as f:
        f.write(ujson.dumps(h5chunks.translate()).encode())

### Accessing single indexed data file

## Data file collection indexing and use

### Creating JSON index files for many data files

In [ ]:
# Create file system to read data files on AWS S3
fs = fsspec.filesystem("s3", skip_instance_cache=True)

# Retrieve list of all data files at desired location
s3_files = fs.glob(S3_PREFIX + "/*/*/*/*.nc")

# Here we prepend the prefix 's3://' to each file path to make a full S3 URI
s3_data_file_paths = sorted(["s3://" + f for f in s3_files])

In [ ]:
print(f"Number of data files found: {len(s3_data_file_paths)}")

In [ ]:
print(s3_data_file_paths[0])

In [ ]:
# Iterate through the list of data file paths and create a JSON index file for each one

temp_dir = os.path.join(LOCAL_DIR, "demo_index_jsons") # Local directory to write all the JSON index files to

temp_dir_path = Path(temp_dir)
if temp_dir_path.is_dir():
    shutil.rmtree(temp_dir)
temp_dir_path.mkdir()

# Optional keyword arguments for fs.open (see fsspec doco for more information)
so = {
    "mode" : "rb",
    "default_fill_cache" : False,
    "default_cache_type" : "first"
}

def generate_json_reference(uri: str, temp_dir: str) -> str:
    """
    Generate JSON index file for a data file and return the full file path to said JSON index file.
    """
    with fs.open(uri, **so) as infile:
        h5chunks = SingleHdf5ToZarr(infile, uri, inline_threshold=300)
        fname = uri.split("/")[-1].strip(".nc")
        outf = os.path.join(temp_dir, f"{fname}.json")
        with open(outf, "wb") as f:
            f.write(ujson.dumps(h5chunks.translate()).encode())
        return outf

# Iterate through the list of data files and generate JSON index files for each one. This loop could easily be parallelized or distributed using dask.
output_files = []
for data_file_path in tqdm(s3_data_file_paths):
    outf = generate_json_reference(data_file_path, temp_dir)
    output_files.append(outf)
output_files.sort()

In [ ]:
NUM_TO_PRINT = 3
for i in range(min(len(output_files), NUM_TO_PRINT)):
    print(output_files[i])

#### Create combined JSON file to represent entire data product

In [ ]:
# Combine individual JSON index files into a single consolidated JSON file.

# In this example, each NetCDF file contains different values on the time dimension, but they all share the same spatial dimensions.
# NOTE: 'crs' is mentioned as a dimension due to how projection information can be stored in NetCDF files.

combined_index_file_path = os.path.join(LOCAL_DIR, "combined_kerchunk.json")

mzz = MultiZarrToZarr(
    output_files,
    concat_dims=["time"],
    identical_dims=["longitude", "latitude", "crs"],
)

multi_kerchunk = mzz.translate()

with open(combined_index_file_path, "wb") as f:
    f.write(ujson.dumps(multi_kerchunk).encode())

del multi_kerchunk
del mzz

#### Alternately, create Parquet index file system to represent entire data product

In [ ]:
dir_path = Path(os.path.join(LOCAL_DIR, "demo_index_jsons"))
output_files = list(dir_path.rglob('*.json'))

from fsspec.implementations.reference import LazyReferenceMapper

parquet_directory = os.path.join(LOCAL_DIR, "index.parq")
parquet_directory_path = Path(parquet_directory)

if parquet_directory_path.is_dir():
    shutil.rmtree(parquet_directory)
parquet_directory_path.mkdir()

fs = fsspec.filesystem("file") # Am going to write this Parquet file system locally (could be remote if desired)

out = LazyReferenceMapper.create(root=parquet_directory, fs=fs, record_size=1000)

mzz = MultiZarrToZarr(
    output_files,
    remote_protocol = "s3",
    concat_dims = ["time"],
    identical_dims= ["longitude", "latitude", "crs"],
    out=out,
).translate()

out.flush()

del out
del mzz

### Accessing multi data file data product

#### Open using combined JSON index file

In [ ]:
fs = fsspec.filesystem(
    "reference",
    fo = combined_index_file_path, # Index file is local in this case
    remote_protocol = "s3", # Data files are stored on S3
    remote_options= {} # Add authentication to S3 if required in here (see fsspec doco)
)

ds = xr.open_dataset(fs.get_mapper(""), engine="zarr", backend_kwargs=dict(consolidated=False), chunks={'time':12, 'latitude':5, 'longitude':5})

display(ds)

#### Alternately, open using Parquet index file system

In [ ]:
fs = fsspec.filesystem(
    "reference",
    fo = parquet_directory, # Index file is local in this case
    remote_protocol = "s3", # Data files are stored on S3
    remote_options = {} # Add authentication to S3 if required in here (see fsspec doco)
)

ds = xr.open_dataset(fs.get_mapper(""), engine="zarr", backend_kwargs=dict(consolidated=False), chunks={'time':12, 'latitude':5, 'longitude':5})

display(ds)

In [ ]:
%%time

# Perform a temporal drill of one pixel through all available time

ds_drill = ds['b1'].isel(latitude=slice(5,6), longitude=slice(10,11)).squeeze().compute()

display(ds_drill)

In [ ]:
%%time

# Perform a spatial read on one time slice

ds_raster = ds['b1'].isel(time=slice(100,101)).squeeze().compute()

display(ds_raster)

#### Open using a Parquet index file system with different chunking size

In [ ]:
fs = fsspec.filesystem(
    "reference",
    fo = parquet_directory, # Index file is local in this case
    remote_protocol = "s3", # Data files are stored on S3
    remote_options = {} # Add authentication to S3 if required in here (see fsspec doco)
)

# The default spatial chunking in this example is small and is great for minimising download size for temporal drills, but slower for spatial reads.
# But you can easily increase the size of the chunks when you open the dataset which leads to more efficient (fewer, larger reads) data access.
ds = xr.open_dataset(fs.get_mapper(""), engine="zarr", backend_kwargs=dict(consolidated=False), chunks={'time':12, 'latitude':10, 'longitude':10})

display(ds)

In [ ]:
%%time

# Perform a temporal drill of one pixel through all available time

ds_drill = ds['b1'].isel(latitude=slice(5,6), longitude=slice(10,11)).squeeze().compute()

display(ds_drill)

In [ ]:
%%time

# Perform a spatial read on one time slice

ds_raster = ds['b1'].isel(time=slice(100,101)).squeeze().compute()

display(ds_raster)

### HTTPS w/simple authentication (TERN Landscapes Data Services)

In [ ]:
# Example of how to open an fsspec to TERN Landscapes Data Services

API_KEY = "<BYO_API_KEY>" # Ingest and use your TERN API KEY here

index_tern_path = "s3://landscapes-easi-shared/misc/testing/himawari_tern_test/index_tern.parq" # Example path to local or S3 based Parquet index file system

fs = fsspec.filesystem(
    "reference",
    fo=index_tern_path,
    remote_protocol="https",
    remote_options={ "headers": { 'x-api-key' : API_KEY } },
    retries=10,
    timeout=300
)

In [ ]:
ds = xr.open_dataset(fs.get_mapper(""), engine="zarr", backend_kwargs=dict(consolidated=False), chunks={'time':12, 'latitude':5, 'longitude':5})

In [ ]:
display(ds)

In [ ]:
ds_pixel = ds['b1'].isel(latitude=slice(0,1),longitude=slice(0,1)).squeeze()

display(ds_pixel)

In [ ]:
%%time

ds_pixel.compute()